## Análisis de un fondo de inversión

Un fondo de inversión es un vehículo financiero que reúne capital de varios inversores para invertir colectivamente en una diversa cartera de activos, como acciones, bonos, o instrumentos del mercado monetario, gestionado por profesionales.

Las características específicas de cómo y en qué invierte el fondo, incluyendo su estrategia de inversión, objetivos, riesgo, y perfil de retorno, se definen detalladamente en su folleto. Este documento es esencial para que los inversores entiendan la naturaleza del fondo, sus metas y cómo busca alcanzarlas, proporcionando la transparencia necesaria para tomar decisiones informadas de inversión.

Las características de los fondos de inversión se pueden explorar detalladamente en la web de Morningstar https://www.morningstar.com/funds , una plataforma reconocida por ofrecer análisis y datos financieros.



Vamos a usar, por ejemplo el fondo [JPMorgan Funds - US Small Cap Growth Fund A (acc) - EUR](https://www.morningstar.es/es/funds/snapshot/snapshot.aspx?id=F000002ETE)

En Morningstar, se destaca el uso del "Style Box", una herramienta visual que clasifica los fondos de inversión según su tamaño (capitalización de mercado) y estilo (valor, mezcla, crecimiento), proporcionando una comprensión rápida de la naturaleza de la inversión del fondo.

<p align="center">
  <img src="../imgs/descripcion fondo.PNG" alt="Fama French Web" style="max-width:70%;">
</p>

Además, Morningstar ofrece información sobre las industrias en las que invierte cada fondo, permitiendo a los inversores analizar la diversificación sectorial y la alineación del fondo con sus preferencias o expectativas de inversión. Esta información es crucial para evaluar la adecuación de un fondo a una estrategia de inversión particular.

Primero cargamos los datos históricos del fondo de inversión.

In [31]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import yfinance as yf

In [32]:
filepath = 'https://raw.githubusercontent.com/alfonso-santos/microcredencial-carteras-python-2023/main/Tema_5_APT/data/Datos%20hist%C3%B3ricos%200P0000IVJL.xlsx'

fund_data_df = pd.read_excel(filepath, 
                               index_col=0)

fund_data_df = fund_data_df[[' var.']]
fund_data_df.columns = ['Return']
fund_data_df /= 100
fund_data_df.head(5)

,Return
Fecha,
2024-02-01,0.0617
2024-01-01,0.0039
2023-12-01,0.1073
2023-11-01,0.0768
2023-10-01,-0.1165


Cargamos los datos de los factores de Fama-French

In [33]:
filepath = 'https://raw.githubusercontent.com/alfonso-santos/microcredencial-carteras-python-2023/main/Tema_5_APT/data/F-F_Research_Data_Factors_monthly.csv'

ff_factors = pd.read_csv(filepath, index_col=0, parse_dates=True)
ff_factors = ff_factors / 100

ff_factors

,Mkt-RF,SMB,HML,RF
1926-07-01,0.0296,-0.0256,-0.0243,0.0022
1926-08-01,0.0264,-0.0117,0.0382,0.0025
1926-09-01,0.0036,-0.0140,0.0013,0.0023
1926-10-01,-0.0324,-0.0009,0.0070,0.0032
1926-11-01,0.0253,-0.0010,-0.0051,0.0031
...,...,...,...,...
2023-07-01,0.0321,0.0208,0.0411,0.0045
2023-08-01,-0.0239,-0.0316,-0.0106,0.0045
2023-09-01,-0.0524,-0.0251,0.0152,0.0043
2023-10-01,-0.0319,-0.0387,0.0019,0.0047


In [34]:
# Creamos el dataframe con os factores y los retornos del fondo

datos_df = pd.concat([ff_factors, fund_data_df], axis=1, join='inner')
datos_df.columns = list(ff_factors.columns) + ['Return']
datos_df.head(5)

,Mkt-RF,SMB,HML,RF,Return
2018-12-01,-0.0957,-0.0238,-0.0184,0.0020,-0.1171
2019-01-01,0.0840,0.0290,-0.0046,0.0021,0.1159
2019-02-01,0.0340,0.0205,-0.0267,0.0018,0.1167
2019-03-01,0.0110,-0.0305,-0.0417,0.0019,0.0173
2019-04-01,0.0397,-0.0174,0.0215,0.0021,0.0423


Hacemos la regresión lineal entre los retornos del fondo y los factores

In [35]:
Y = datos_df.loc[:, 'Return']-datos_df['RF']
X = datos_df[['Mkt-RF', 'SMB', 'HML']]

X = sm.add_constant(X)

modelo = sm.OLS(Y, X).fit()

print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.776
Model:                            OLS   Adj. R-squared:                  0.764
Method:                 Least Squares   F-statistic:                     64.78
Date:                Tue, 09 Apr 2024   Prob (F-statistic):           3.31e-18
Time:                        14:30:18   Log-Likelihood:                 117.79
No. Observations:                  60   AIC:                            -227.6
Df Residuals:                      56   BIC:                            -219.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0025      0.005     -0.537      0.5

Los resultados de la regresión lineal utilizando el modelo de tres factores de Fama-French indican lo siguiente:

**R-squared (R²):** Con un valor de 0.776, muestra que el modelo explica el 77.6% de la variabilidad en los retornos del fondo, evidenciando una fuerte correlación entre los retornos del fondo y los factores de riesgo del mercado.

**Coeficientes y sus p-valores:**

- **const:** -0.0025 con p-valor de 0.594, lo que indica que el intercepto no es estadísticamente significativo.
- **Mkt-RF:** 0.8587 con p-valor < 0.001, lo que muestra una alta sensibilidad a los movimientos del mercado y es estadísticamente significativo.
- **SMB:** 0.9536 con p-valor < 0.001, indicando una fuerte preferencia por empresas de pequeña capitalización y es estadísticamente significativo.
- **HML:** -0.2618 con p-valor de 0.010, sugiriendo una leve preferencia a las acciones de crecimiento y es estadísticamente significativo.

**Diagnósticos de los residuos:** Los valores Omnibus y Jarque-Bera, junto con sus respectivos p-valores, indican no hay evidencia significativa de no normalidad en los residuos.

Este análisis sugiere que los factores de riesgo del mercado, las empresas de pequeña capitalización y una leve aversión hacia las acciones de valor tienen una influencia significativa en los retornos del fondo, con evidencia estadística robusta para cada factor.

**Nos bajamos los factores por industria** En este caso usamos el modelo con 5 factores.

Definimos una función para parsear las fechas ya que están en el formato 192607, 192608, 192609 ...

In [36]:
# Definir una función para analizar las fechas en el formato específico
dateparse = lambda x: pd.to_datetime(x, format='%Y%m')

filepath = 'https://raw.githubusercontent.com/alfonso-santos/microcredencial-carteras-python-2023/main/Tema_5_APT/data/5_Industry_Portfolios.CSV'

# Leer el archivo CSV con el analizador de fechas personalizado
industries_ff_df = pd.read_csv(filepath, 
                               index_col=0, 
                               parse_dates=True, 
                               date_parser=dateparse)

industries_ff_df = industries_ff_df / 100
industries_ff_df

/tmp/ipykernel_46745/2193698918.py:7: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  industries_ff_df = pd.read_csv(filepath,


,Cnsmr,Manuf,HiTec,Hlth,Other
1926-07-01,0.0543,0.0273,0.0183,0.0177,0.0213
1926-08-01,0.0276,0.0233,0.0241,0.0425,0.0435
1926-09-01,0.0216,-0.0044,0.0106,0.0069,0.0029
1926-10-01,-0.0390,-0.0242,-0.0226,-0.0057,-0.0284
1926-11-01,0.0370,0.0250,0.0307,0.0542,0.0211
...,...,...,...,...,...
2023-08-01,-0.0182,-0.0182,-0.0158,-0.0022,-0.0335
2023-09-01,-0.0494,-0.0395,-0.0582,-0.0471,-0.0341
2023-10-01,-0.0343,-0.0305,-0.0168,-0.0458,-0.0253
2023-11-01,0.0788,0.0522,0.1165,0.0587,0.1041


In [37]:
#Generamos un df con los factores y los retornos del fondo

datos_ind_df = pd.concat([industries_ff_df, fund_data_df], axis=1, join='inner')
datos_ind_df.columns = list(industries_ff_df.columns) + ['Fund']
datos_ind_df = pd.concat([datos_ind_df,ff_factors['RF']], axis=1, join='inner')
datos_ind_df.head(5)

,Cnsmr,Manuf,HiTec,Hlth,Other,Fund,RF
2018-12-01,-0.0988,-0.0906,-0.0823,-0.0825,-0.1094,-0.1171,0.0020
2019-01-01,0.0810,0.0896,0.0870,0.0532,0.0980,0.1159,0.0021
2019-02-01,0.0109,0.0416,0.0539,0.0330,0.0296,0.1167,0.0018
2019-03-01,0.0285,0.0072,0.0332,0.0050,-0.0147,0.0173,0.0019
2019-04-01,0.0411,0.0243,0.0587,-0.0316,0.0663,0.0423,0.0021


Hacemos la regresión entre los retornos del fondo y las industrias

In [38]:
Y = datos_ind_df.loc[:, 'Fund']-datos_ind_df['RF']
X = datos_ind_df[['Cnsmr', 'Manuf', 'HiTec', 'Hlth ', 'Other']]

X = sm.add_constant(X)

modelo = sm.OLS(Y, X).fit()

print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.670
Model:                            OLS   Adj. R-squared:                  0.640
Method:                 Least Squares   F-statistic:                     21.97
Date:                Tue, 09 Apr 2024   Prob (F-statistic):           6.14e-12
Time:                        14:30:18   Log-Likelihood:                 106.17
No. Observations:                  60   AIC:                            -200.3
Df Residuals:                      54   BIC:                            -187.8
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0056      0.006     -0.954      0.3

Análisis de los resultados de la regresión lineal con cinco variables sectoriales:

- **R-squared (R²):** 0.670, indicando que el modelo explica el 67% de la variabilidad en los retornos.

- **Coeficientes y p-valores:**
  - **const:** -0.0056 (p-valor: 0.344), no significativo.
  - **Cnsmr (Consumo):** 0.8511 (p-valor: 0.004), indicando una fuerte influencia positiva del sector de consumo en los retornos del fondo y es estadísticamente significativo.
  - **Manuf (Manufactura):** -0.3520 (p-valor: 0.183), no significativo.
  - **HiTec (Alta Tecnología):** -0.0932 (p-valor: 0.712), no significativo.
  - **Hlth (Salud):** 0.1950 (p-valor: 0.332), no significativo.
  - **Other (Otros):** 0.4368 (p-valor: 0.100), marginalmente no significativo.

- **Diagnósticos de residuos:** Sin evidencia de problemas de normalidad (Omnibus: 0.170, p-valor: 0.919; Jarque-Bera: 0.022, p-valor: 0.989).

El sector de consumo muestra una influencia positiva y significativa en los retornos del fondo, mientras que los efectos de otros sectores no son estadísticamente significativos.


La web de Morininstar también nos da los **10 activos en los que más invierte el fondo**. Podemos ver si alguno de ellos explica los retornos del fondo.

<p align="center">
  <img src="../imgs/activos_fondo.PNG" alt="Fama French Web" style="max-width:30%;">
</p>

Elegimos unos cuántos activos, nos bajamos datos de ellos y hacemos una regresión con los retornos del fondo

In [39]:
# Diccionario con los activos elegidos
activos = { 'Super Micro': 'SMCI', 'e.l.f. Beauty': '0EF.F', 'Comfort Systems USA': 'FIX', 'AAON': 'AAON', 'Natera': 'NTRA'}

precios_act = yf.download(list(activos.values()), interval='1mo')['Adj Close']

retornos_act = np.log(precios_act).diff().dropna()

[*********************100%%**********************]  5 of 5 completed


In [40]:
datos_df_act = pd.concat([retornos_act, datos_ind_df.loc[:, 'RF']], axis=1, join='inner')
datos_df_act.columns = list(retornos_act.columns) + ['RF']
datos_df_act.head(5)

,0EF.F,AAON,FIX,NTRA,SMCI,RF
2018-12-01,-0.389861,-0.074941,-0.185431,-0.215666,-0.076693,0.0020
2019-01-01,0.135238,0.052234,0.093686,-0.026862,0.090026,0.0021
2019-02-01,-0.021714,0.076078,0.111346,0.153204,0.236040,0.0018
2019-03-01,0.134446,0.147174,-0.023207,0.263723,0.099959,0.0019
2019-04-01,0.171783,0.083667,0.033946,-0.076050,0.059260,0.0021


In [41]:
Y = datos_ind_df.loc[:, 'Fund']-datos_ind_df['RF']
X = datos_df_act[activos.values()]

X = sm.add_constant(X)

modelo = sm.OLS(Y, X).fit()

print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.445
Model:                            OLS   Adj. R-squared:                  0.394
Method:                 Least Squares   F-statistic:                     8.659
Date:                Tue, 09 Apr 2024   Prob (F-statistic):           4.47e-06
Time:                        14:30:18   Log-Likelihood:                 90.530
No. Observations:                  60   AIC:                            -169.1
Df Residuals:                      54   BIC:                            -156.5
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0100      0.008     -1.299      0.1